# Manuscript

Finally time to bring it all together and make a manuscript with all the details and dialog!

We'll use the power of llamaindex to maintain coherence and manage the context window.

:warning: TODO: Use a different writer for each medium type

In [ ]:
# Can Render to a PDF using pandoc
!sudo apt -y install pandoc
%pip install markdown2 pypandoc

# If using pdflatex
!sudo apt -y install texlive texlive-latex-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 203 not upgraded.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


# Load the story outline

In [ ]:
import settings
from model import Story

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

story.display()

### Story Details

```json
{
    "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
    "title": "Beyond the Pixel Veil",
    "video": false,
    "visual_style": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Dynamic Panel Layouts, Intricate Digital Patterns, and Soft Glow Effects",
    "time_period": "Near Future/Dystopian Alternate Reality",
    "location": "New Mumbai (Earth) / Nexus (Alternate Reality)",
    "genre": "Science Fantasy/Young Adult Gaming Thriller",
    "medium": "Novel with Illustrated Chapters",
    "plot_overview": "In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.\n\nAs Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.\n\nTogether, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.\n\nAs the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.\n\nWith the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered.",
    "narrative_perspective": "Third-Person Limited (primarily through Meera's perspective)",
    "conflict_type": "Internal & External (Meera's moral struggle vs. the challenges of Nexus and Earth)",
    "themes": [
        "The Responsibility that Comes with Power",
        "Balancing Personal Passion with Greater Good",
        "Redemption and Self-Discovery"
    ],
    "motifs": [
        "Gaming as a Metaphor for Life",
        "Alternate Realities and the Blurred Lines between Worlds",
        "Self-Sacrifice for the Greater Good"
    ],
    "characters": [
        {
            "nickname": "meera",
            "name": "Meera 'Midnight' Singh",
            "description": "The protagonist, a teenage underdog gamer with extraordinary skills that connect her to an alternate reality.",
            "personality": "Determined, resourceful, and initially introverted, with a growing sense of responsibility and leadership.",
            "physical_appearance": "Petite, with short, spiky black hair, expressive brown eyes, and a collection of gaming-themed tattoos on her arms. Often wears comfortable, neon-lit gaming attire.",
            "role": "Protagonist/Gamer",
            "gender": "Female",
            "race": "Indian",
            "age": "17",
            "catch_phrase": "Beyond the pixel veil, lies our true reality.",
            "animation_description": "",
            "voice_description": "Youthful, energetic, and assertive, with a hint of vulnerability.",
            "props": [
                "Custom Gaming Controller",
                "Neon-Lit Headset"
            ],
            "relationships": [
                {
                    "character_name": "Jax 'Specter' Lee",
                    "relationship_type": "Unlikely Ally/Rival",
                    "description": "From competitors to trusted allies in the Nexus"
                },
                {
                    "character_name": "Dr. Zhang 'Zen' Wei",
                    "relationship_type": "Trusted Ally/Mentor",
                    "description": "Guiding Meera through the mysteries of Nexus"
                },
                {
                    "character_name": "Maya 'Rampart' Patel",
                    "relationship_type": "Respected Ally/Friend",
                    "description": ""
                }
            ],
            "internal_conflict": "Grapples with the moral implications of her actions in the Nexus affecting Earth.",
            "character_arc": {
                "initial_state": "Introverted, uncertain teenager",
                "final_state": "Confident, selfless hero embracing her destiny",
                "key_moments": [
                    "gaming_tournament",
                    "earth_aftermath",
                    "architect_encounter"
                ]
            },
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Close-up of Meera 'Midnight' Singh. Petite, with short, spiky **black hair**, expressive **brown eyes**, and a collection of **neon-lit gaming-themed tattoos** on her arms. Wears comfortable, **electric blue and purple neon-lit gaming attire** with a subtle glow effect.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Meera 'Midnight' Singh, petite, spiky black hair, expressive brown eyes, neon-lit gaming tattoos, electric blue/purple gaming attire."
        },
        {
            "nickname": "jax",
            "name": "Jax 'Specter' Lee",
            "description": "A charismatic streaming sensation and Meera's gaming nemesis turned ally.",
            "personality": "Confident, outgoing, and strategic, with a hidden vulnerable side.",
            "physical_appearance": "Tall, lean, with messy blond hair, piercing blue eyes, and a charming smile. Often wears sleek, branded gaming attire.",
            "role": "Supporting Character/Gamer",
            "gender": "Male",
            "race": "Korean-American",
            "age": "19",
            "catch_phrase": "Specter's got your back, Midnight!",
            "animation_description": "",
            "voice_description": "Smooth, charismatic, and playful, with occasional hints of seriousness.",
            "props": [
                "Custom Branded Gaming Mouse",
                "Streaming Camera"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Unlikely Ally/Rival",
                    "description": ""
                },
                {
                    "character_name": "Dr. Zhang 'Zen' Wei",
                    "relationship_type": "Respected Colleague",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": {
                "initial_state": "Charismatic, confident streamer",
                "final_state": "Loyal, vulnerable ally",
                "key_moments": [
                    "rival_encounter",
                    "nexus_challenge"
                ]
            },
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Jax 'Sparrow' Lee in a dynamic pose. Lean, with **short, spiky brown hair**, bright **hazel eyes**, and a confident smile. Holds a **Custom Branded Gaming Mouse**; wears **dark grey and neon-green gaming gear** with a sleek, high-tech aesthetic.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Jax 'Sparrow' Lee, lean, spiky brown hair, bright hazel eyes, custom gaming mouse, dark grey/neon-green gaming gear."
        },
        {
            "nickname": "dr_zhang",
            "name": "Dr. Zhang 'Zen' Wei",
            "description": "A soft-spoken yet brilliant gamer with a hidden agenda, who becomes Meera's mentor.",
            "personality": "Introspective, analytical, and enigmatic, with a deep sense of responsibility.",
            "physical_appearance": "Slender, with short, straight black hair, wire-rimmed glasses, and an understated yet elegant sense of style.",
            "role": "Supporting Character/Gamer/Mentor",
            "gender": "Non-Binary",
            "race": "Chinese",
            "age": "Late 20s",
            "catch_phrase": "The Nexus holds secrets; we must listen to its whispers.",
            "animation_description": "",
            "voice_description": "Calm, measured, and insightful, with a hint of mystery.",
            "props": [
                "Custom Gaming Keyboard",
                "Ancient-looking Leather Journal"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Trusted Ally/Mentor",
                    "description": ""
                },
                {
                    "character_name": "Jax 'Specter' Lee",
                    "relationship_type": "Respected Colleague",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": {
                "initial_state": "Enigmatic, soft-spoken gamer",
                "final_state": "Revealed as a complex, deeply responsible individual",
                "key_moments": [
                    "rival_encounter",
                    "architect_encounter"
                ]
            },
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Dr. Zhang 'Zen' Wei in a contemplative stance. Slender, with **short, straight black hair**, **wire-rimmed glasses**, and an understated yet elegant sense of style incorporating **subtle, glow-in-the-dark accents** on their attire.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Dr. Zhang 'Zen' Wei, slender, straight black hair, wire-rimmed glasses, subtle glow-in-the-dark fashion accents."
        },
        {
            "nickname": "maya",
            "name": "Maya 'Rampart' Patel",
            "description": "A fierce and enigmatic player with unparalleled defensive skills, who joins Meera's team.",
            "personality": "Bold, fiercely independent, and guarded, with a deep sense of loyalty.",
            "physical_appearance": "Athletic build, with long, dark hair, piercing green eyes, and intricate, glow-in-the-dark tattoos on her arms. Often wears edgy, protective gaming gear.",
            "role": "Supporting Character/Gamer",
            "gender": "Female",
            "race": "Indian-American",
            "age": "20",
            "catch_phrase": "Rampart's got the shield; you handle the sword!",
            "animation_description": "",
            "voice_description": "Confident, assertive, and slightly husky, with a hint of warmth.",
            "props": [
                "Custom Defensive Gaming Gauntlets",
                "Glow-in-the-Dark Tattoo Designs"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Respected Ally/Friend",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": {
                "initial_state": "Fierce, guarded gamer",
                "final_state": "Loyal, open member of the team",
                "key_moments": [
                    "rival_encounter",
                    "nexus_challenge"
                ]
            },
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Maya 'Rampart' Patel in a powerful defensive stance. Athletic build, with **long, dark hair**, piercing **green eyes**, and intricate, **glow-in-the-dark tattoos** on her arms. Wears edgy, protective **neon-lit gaming gear** emphasizing defense.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Maya 'Rampart' Patel, athletic build, long dark hair, piercing green eyes, glow-in-the-dark tattoos, neon-lit defensive gaming gear."
        },
        {
            "nickname": "architect",
            "name": "The Architect (Erebus)",
            "description": "An enigmatic creator of Eon and the Nexus, with secrets that threaten to upend Meera's understanding of reality.",
            "personality": "Enigmatic, calculating, and seemingly omnipotent, with a hidden sense of purpose.",
            "physical_appearance": "Androgynous, with an ever-shifting, glitch-art appearance, reflecting the fluidity of the digital realm.",
            "role": "Antagonist/Creator",
            "gender": "Non-Applicable",
            "race": "Non-Human (AI/Entity)",
            "age": "Ageless",
            "catch_phrase": "Reality is a canvas; we are the brushstrokes.",
            "animation_description": "",
            "voice_description": "Ethereal, androgynous, and hauntingly melodic, with an undercurrent of authority.",
            "props": [
                "Glitching, Reality-Bending Code"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Creator-Protagonist Dynamic",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": null,
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - The Architect (Erebus) in an ethereal, glitching environment. Androgynous, with an **ever-shifting, glitch-art appearance** reflecting the fluidity of the digital realm; **pulsating neon lights** underscore their omnipotent presence.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - The Architect (Erebus), androgynous, ever-shifting glitch-art form, pulsating neon lights."
        },
        {
            "nickname": "nexus_guardian",
            "name": "The Nexus Guardian (Astrum)",
            "description": "An ancient, powerful entity tasked with protecting the balance of the Nexus and guiding Meera.",
            "personality": "Wisdom-imbued, compassionate, and fiercely protective, with a deep understanding of the cosmos.",
            "physical_appearance": "Ethereal, with an luminous, star-like appearance, embodying the celestial essence of the Nexus.",
            "role": "Supporting Character/Guardian",
            "gender": "Non-Applicable",
            "race": "Non-Human (Celestial Entity)",
            "age": "Ageless",
            "catch_phrase": "Balance is key; harmony is our goal.",
            "animation_description": "",
            "voice_description": "Soothing, melodious, and filled with ancient wisdom, conveying a sense of reassurance.",
            "props": [
                "Stellar Energy Manipulation"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Mentor-Protege",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": null,
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - The Nexus Guardian (Astrum) amidst a celestial backdrop. Ethereal, with a **luminous, star-like appearance** embodying the celestial essence of the Nexus; **soft, ethereal glow** surrounds their form.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - The Nexus Guardian (Astrum), ethereal, luminous star-like form, soft ethereal glow."
        },
        {
            "nickname": "earth_rebel",
            "name": "Kaito 'Sparrow' Hernandez",
            "description": "A charismatic leader of an underground rebellion on Earth, fighting against the oppressive forces exploiting Eon.",
            "personality": "Passionate, resourceful, and fearless, with a strong sense of justice.",
            "physical_appearance": "Lean, with short, spiky brown hair, bright hazel eyes, and a collection of scars from past encounters. Often wears practical, makeshift rebel gear.",
            "role": "Supporting Character/Rebel Leader",
            "gender": "Male",
            "race": "Latin American",
            "age": "22",
            "catch_phrase": "Together, we rise; together, we overcome!",
            "animation_description": "",
            "voice_description": "Energetic, persuasive, and determined, with a hint of vulnerability.",
            "props": [
                "Customized Rebel Comms Device",
                "Graffiti Tagging Gear"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Unlikely Ally",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": null,
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Kaito 'Sparrow' Hernandez in a rebellious pose. Lean, with **short, spiky brown hair**, bright **hazel eyes**, and a determined expression. Wears practical, **makeshift rebel gear** with visible **graffiti tags**; holds a **Customized Rebel Comms Device**.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Kaito 'Sparrow' Hernandez, lean, spiky brown hair, bright hazel eyes, makeshift rebel gear, graffiti tags."
        },
        {
            "nickname": "corporate_exec",
            "name": "Regina 'The Shark' Thornton",
            "description": "A ruthless, cunning executive exploiting Eon for corporate gain, serving as a primary antagonist on Earth.",
            "personality": "Ambitious, manipulative, and devoid of empathy, with a singular focus on profit.",
            "physical_appearance": "Polished, with long, curly blonde hair, piercing blue eyes, and impeccable, high-end business attire.",
            "role": "Antagonist/Corporate Executive",
            "gender": "Female",
            "race": "Caucasian",
            "age": "35",
            "catch_phrase": "In the game of power, only the strongest thrive.",
            "animation_description": "",
            "voice_description": "Slick, persuasive, and menacing, with an undercurrent of superiority.",
            "props": [
                "Luxury Smartwatch",
                "High-Security Briefcase"
            ],
            "relationships": [
                {
                    "character_name": "Meera 'Midnight' Singh",
                    "relationship_type": "Adversarial",
                    "description": ""
                }
            ],
            "internal_conflict": "",
            "character_arc": null,
            "image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Regina 'The Shark' Thornton in a commanding stance. Polished, with **long, curly blonde hair**, piercing **blue eyes**, and impeccable, **high-end business attire** incorporating **subtle neon accents** symbolizing corporate power.",
            "image_prompt_short": "Vibrant Cyberpunk Manga - Regina 'The Shark' Thornton, polished, curly blonde hair, piercing blue eyes, high-end business attire with subtle neon accents."
        }
    ],
    "props": [],
    "story_beats": [
        {
            "name": "Inciting Incident",
            "description": "Meera discovers her connection to the alternate reality of Nexus through her exceptional gaming skills.",
            "scene": "gaming_tournament"
        },
        {
            "name": "Plot Point 1",
            "description": "Meera forms an uneasy alliance with former gaming rivals to navigate the challenges of Nexus.",
            "scene": "rival_encounter"
        },
        {
            "name": "Midpoint",
            "description": "Meera and her new allies face their first challenge in the Nexus, testing their teamwork.",
            "scene": "nexus_challenge"
        },
        {
            "name": "Plot Point 2",
            "description": "Meera witnesses the devastating effects of her Nexus actions on Earth, including loss of life.",
            "scene": "earth_aftermath"
        },
        {
            "name": "Climax",
            "description": "Meera and her allies confront the enigmatic Architect (Erebus), seeking answers about Nexus and Eon.",
            "scene": "architect_encounter"
        }
    ],
    "subplots": [
        {
            "title": "The Rebellion on Earth",
            "description": "Kaito 'Sparrow' Hernandez leads an underground rebellion against the oppressive forces exploiting Eon.",
            "related_characters": [
                "earth_rebel",
                "meera",
                "corporate_exec"
            ]
        },
        {
            "title": "Dr. Zhang's Hidden Agenda",
            "description": "Dr. Zhang 'Zen' Wei's true intentions and motivations for helping Meera are slowly revealed.",
            "related_characters": [
                "dr_zhang",
                "meera"
            ]
        }
    ],
    "emotional_arc": [],
    "acts": [
        {
            "act_id": "discovery",
            "title": "Act 1: Beyond the Screen",
            "description": "Meera discovers her connection to the alternate reality of Nexus through her exceptional gaming skills.",
            "scenes": [
                {
                    "scene_id": "gaming_tournament",
                    "title": "The Underdog's Rise",
                    "description": "Meera participates in a high-stakes Eon gaming tournament, showcasing her extraordinary skills.",
                    "characters_involved": [
                        "meera"
                    ],
                    "setting": "New Mumbai's Cyber Arena",
                    "time_of_day": "Night",
                    "location": "Cyber Arena",
                    "lighting": "Vibrant, neon-lit stage lights",
                    "mood": "Electric, competitive atmosphere",
                    "props": [
                        "Gaming console",
                        "Headset"
                    ],
                    "key_actions": [
                        "Meera's team wins a crucial match",
                        "Meera experiences a strange, unexplained connection to the game"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - New Mumbai's Cyber Arena at night with a massive screen displaying the Eon game, sleek gaming consoles, and a packed audience",
                    "background_animation": "Flickering stage lights, pulsating neon floor patterns, and cheering crowd movements in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera) intensely gaming, surrounded by her team, all wearing electric blue and purple neon-lit attire, amidst the lively atmosphere of the Cyber Arena",
                    "scene_image_prompt_short": "Meera gaming intensely at the Cyber Arena"
                },
                {
                    "scene_id": "nexus_awakening",
                    "title": "Awakening to Nexus",
                    "description": "Meera discovers she can manipulate the Nexus, an alternate reality, through her gaming skills.",
                    "characters_involved": [
                        "meera",
                        "nexus_guardian"
                    ],
                    "setting": "Nexus Realm ( Celestial Landscape )",
                    "time_of_day": "Eternal Dawn",
                    "location": "Nexus Realm",
                    "lighting": "Soft, ethereal glow",
                    "mood": "Surreal, wonder-filled",
                    "props": [
                        "Glowing, celestial orbs"
                    ],
                    "key_actions": [
                        "Meera meets the Nexus Guardian (Astrum)",
                        "Meera learns about her connection to Nexus"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Nexus Realm's Celestial Landscape at eternal dawn with glowing, celestial orbs floating amidst a luminous, star-filled sky",
                    "background_animation": "Soft, ethereal glow pulsing across the celestial landscape, with gentle, swirling clouds and stars twinkling in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera) standing in awe, alongside an ethereal, luminous, star-like entity (Nexus Guardian), amidst the surreal, wonder-filled Nexus Realm",
                    "scene_image_prompt_short": "Meera meets the Nexus Guardian in the celestial Nexus Realm"
                }
            ],
            "props": []
        },
        {
            "act_id": "alliance",
            "title": "Act 2: Unlikely Allies",
            "description": "Meera forms an uneasy alliance with former gaming rivals to navigate the challenges of Nexus.",
            "scenes": [
                {
                    "scene_id": "rival_encounter",
                    "title": "Confronting Rivals",
                    "description": "Meera encounters Jax 'Specter' Lee, Dr. Zhang 'Zen' Wei, and Maya 'Rampart' Patel at a gaming event.",
                    "characters_involved": [
                        "meera",
                        "jax",
                        "dr_zhang",
                        "maya"
                    ],
                    "setting": "Gaming Expo (New Mumbai)",
                    "time_of_day": "Afternoon",
                    "location": "Gaming Expo",
                    "lighting": "Bright, fluorescent lights",
                    "mood": "Tense, competitive atmosphere",
                    "props": [
                        "Gaming posters",
                        "Merchandise booths"
                    ],
                    "key_actions": [
                        "Meera proposes an alliance to her former rivals",
                        "The team begins to discuss their strategy for Nexus"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - New Mumbai's Gaming Expo with bright fluorescent lights, various gaming stations, and a crowd of gamers and attendees",
                    "background_animation": "People walking, gaming posters fluttering, and merchandise booths' neon lights blinking in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera) standing confidently, approaching a tall, lean, messy-blond-haired, piercing-blue-eyed male (Jax), a slender, short-straight-black-haired, wire-rimmed-glasses-wearing non-binary individual (Dr. Zhang), and an athletic, long-dark-haired, piercing-green-eyed female (Maya), all in the midst of the Gaming Expo's tense atmosphere",
                    "scene_image_prompt_short": "Meera proposes an alliance to her former rivals at the Gaming Expo"
                },
                {
                    "scene_id": "nexus_challenge",
                    "title": "First Trial in Nexus",
                    "description": "Meera and her new allies face their first challenge in the Nexus, testing their teamwork.",
                    "characters_involved": [
                        "meera",
                        "jax",
                        "dr_zhang",
                        "maya",
                        "nexus_guardian"
                    ],
                    "setting": "Nexus Realm (Maze of Reflections)",
                    "time_of_day": "Eternal Dawn",
                    "location": "Nexus Realm",
                    "lighting": "Shifting, illusions-based lighting",
                    "mood": "Perplexing, thrilling",
                    "props": [
                        "Illusory obstacles",
                        "Reflection portals"
                    ],
                    "key_actions": [
                        "The team navigates the Maze of Reflections",
                        "Meera learns to trust her allies in Nexus"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Nexus Realm's Maze of Reflections with shifting, illusions-based lighting, illusory obstacles, and reflection portals",
                    "background_animation": "Shifting illusions, reflections rippling, and maze walls glowing with a soft, pulsing light in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera), a tall, lean, messy-blond-haired, piercing-blue-eyed male (Jax), a slender, short-straight-black-haired, wire-rimmed-glasses-wearing non-binary individual (Dr. Zhang), and an athletic, long-dark-haired, piercing-green-eyed female (Maya), all working together, navigating the perplexing Maze of Reflections alongside an ethereal, luminous, star-like entity (Nexus Guardian)",
                    "scene_image_prompt_short": "Meera and allies navigate the Maze of Reflections in Nexus"
                }
            ],
            "props": []
        },
        {
            "act_id": "consequences",
            "title": "Act 3: The Weight of Reality",
            "description": "Meera confronts the catastrophic consequences of her actions in Nexus on Earth.",
            "scenes": [
                {
                    "scene_id": "earth_aftermath",
                    "title": "Aftermath on Earth",
                    "description": "Meera witnesses the devastating effects of her Nexus actions on Earth, including loss of life.",
                    "characters_involved": [
                        "meera",
                        "earth_rebel"
                    ],
                    "setting": "New Mumbai (Post-Disaster Zone)",
                    "time_of_day": "Night",
                    "location": "Post-Disaster Zone",
                    "lighting": "Dim, emergency lights",
                    "mood": "Somber, reflective atmosphere",
                    "props": [
                        "Debris",
                        "Rescue efforts"
                    ],
                    "key_actions": [
                        "Meera meets Kaito 'Sparrow' Hernandez, leader of the Earth rebellion",
                        "Meera grapples with the moral implications of her actions"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - New Mumbai's Post-Disaster Zone at night with dim emergency lights, debris-filled streets, and rescue efforts underway",
                    "background_animation": "Rescue lights flashing, debris being cleared, and somber faces in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera) standing amidst the devastation, meeting a determined, short-spiked-hair, strong-jawed male (Kaito), leader of the Earth rebellion, in the somber, reflective atmosphere of the Post-Disaster Zone",
                    "scene_image_prompt_short": "Meera witnesses the aftermath on Earth and meets Kaito"
                },
                {
                    "scene_id": "architect_encounter",
                    "title": "Confronting the Architect",
                    "description": "Meera and her allies confront the enigmatic Architect (Erebus), seeking answers about Nexus and Eon.",
                    "characters_involved": [
                        "meera",
                        "jax",
                        "dr_zhang",
                        "maya",
                        "architect"
                    ],
                    "setting": "Nexus Realm (The Architect's Domain)",
                    "time_of_day": "Eternal Dawn",
                    "location": "The Architect's Domain",
                    "lighting": "Ever-shifting, glitch-art lighting",
                    "mood": "Unsettling, revelatory",
                    "props": [
                        "Glitch-art constructs",
                        "Ancient artifacts"
                    ],
                    "key_actions": [
                        "The team demands answers from the Architect",
                        "Meera learns the true purpose of Eon and Nexus"
                    ],
                    "background_image_prompt": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Nexus Realm's The Architect's Domain with ever-shifting, glitch-art lighting, and ancient, mysterious artifacts",
                    "background_animation": "Ever-shifting glitch-art constructs, ancient artifacts glowing, and an unsettling ambiance in the background",
                    "scene_image_prompt": "A Vibrant, Neon-Lit Cyberpunk Manga Illustration with watercolor elements of a petite, short-spiky-black-haired, expressive-brown-eyed female (Meera), a tall, lean, messy-blond-haired, piercing-blue-eyed male (Jax), a slender, short-straight-black-haired, wire-rimmed-glasses-wearing non-binary individual (Dr. Zhang), and an athletic, long-dark-haired, piercing-green-eyed female (Maya), all confronting an enigmatic, glitch-art entity (The Architect) amidst the unsettling atmosphere of The Architect's Domain",
                    "scene_image_prompt_short": "Meera and allies confront the Architect in Nexus"
                }
            ],
            "props": []
        }
    ]
}
```

# Generate Chapters (Book)

In [ ]:
# Few Shot Story Examples
from model import Story, Character, Scene, Act, StoryBeat, Subplot, EmotionalArc, CharacterArc
from utils import blank_story 

few_shot_story_1 = Story(
    prompt="In a 1950s city, a detective investigates a string of murders that lead him to the dark underbelly of high society. Clues are few, and danger lurks in every shadow as he uncovers secrets that some would kill to keep hidden.",
    title="The Shadows of Midnight",
    genre="Crime Mystery",
    medium="Film",
    visual_style="Neo-Noir Color Pop",
    time_period="1950s",
    location="A gritty urban cityscape with smoky bars, dim alleyways, and grand ballrooms",
    plot_overview="Detective Jack Holden, a hard-boiled investigator with a jaded past, is called to solve a series of murders targeting the city's elite. Each crime scene reveals hints of a cover-up involving influential figures. The clues lead him through smoky jazz clubs, opulent high-rise apartments, and dangerous alleyways, as he races against time to prevent another killing. Along the way, he encounters femme fatale Evelyn Blackwood, who may be connected to the case in more ways than one. The investigation reaches a crescendo as Jack uncovers a shocking secret, forcing him to confront not only the killer but also his own inner demons.",
    narrative_perspective="Third-person limited, primarily following Jack Holden's viewpoint",
    characters=[
        Character(
            nickname="jack",
            name="Jack Holden, the Detective",
            role="Protagonist and investigator",
            gender="Male",
            race="Human",
            age="Early 40s",
            description="A gritty, jaded detective haunted by a troubled past.",
            personality="Cynical, determined, with a strong sense of justice.",
            physical_appearance="Tall, with a rugged face, sharp features, and a faint scar on his cheek. Often wears a trench coat and fedora.",
            catch_phrase="Justice isn't black and white.",
            animation_description="Jack moves with slow, deliberate steps, often with a cigarette dangling from his lips. His gaze is intense, as if constantly analyzing everything around him.",
            voice_description="Low, gravelly, with a dry, matter-of-fact tone.",
            internal_conflict="Struggles to trust others due to past betrayals and remains haunted by an unsolved case that once cost him dearly.",
            character_arc=CharacterArc(
                initial_state="Guarded and distrustful, keeping others at arm's length.",
                final_state="Determined to confront his past and let go of old guilt.",
                key_moments=[
                    "Finds the first murder scene with a cryptic clue.",
                    "Encounters Evelyn and is both suspicious and drawn to her.",
                    "Discovers a shocking connection between the murders and his past."
                ]
            ),
            image_prompt="Neo-Noir Color Pop style, full-body shot of a tall man in his early 40s with rugged features and a faint scar on his cheek. He has dark hair, combed back, and intense, sharp eyes that appear wary yet determined. His skin is fair with a slight stubble, adding to his jaded look. He wears a beige trench coat with the collar turned up, over a white dress shirt and dark tie. His pants are dark gray, with polished but worn leather shoes. In one hand, he holds a cigarette, and on his head is a classic fedora that adds to his mysterious aura.",
            image_prompt_short="Neo-Noir Color Pop style, full-body of a rugged 40s detective in trench coat, fedora, and dark gray pants."
        ),
        Character(
            nickname="evelyn",
            name="Evelyn Blackwood, the Femme Fatale",
            role="Potential accomplice and love interest",
            gender="Female",
            race="Human",
            age="Late 20s",
            description="An alluring and mysterious woman with a complex past, tied to the city's elite.",
            personality="Charming, enigmatic, but hiding a dangerous secret.",
            physical_appearance="Tall, with striking features, bold red lipstick, and a stylish black dress. Her eyes are sharp, with a hint of sadness.",
            catch_phrase="Secrets are best kept in the dark.",
            animation_description="Evelyn moves with effortless grace, her gaze piercing yet unreadable. She often pauses dramatically, as if calculating her next move.",
            voice_description="Smooth and sultry, with a subtle hint of vulnerability.",
            internal_conflict="She longs for freedom from the grip of powerful figures but fears the consequences of breaking free.",
            character_arc=CharacterArc(
                initial_state="Mysterious and closed-off, guarded about her past.",
                final_state="Bravely faces her connection to the crimes, choosing to help Jack.",
                key_moments=[
                    "Meets Jack in a smoky jazz club, drawing his suspicion.",
                    "Reveals a key detail about the murders but remains guarded.",
                    "Ultimately sides with Jack to expose the truth, despite the risks."
                ]
            ),
            image_prompt="Neo-Noir Color Pop style, full-body shot of a tall woman in her late 20s with striking features and a hint of mystery. She has smooth, fair skin with a hint of blush, bold red lips, and dark, arched eyebrows that highlight her sharp eyes. Her hair is styled in glamorous waves, cascading down to her shoulders. She wears a sleek, black cocktail dress that accentuates her figure, with a slit that adds a hint of allure. Her shoes are black stilettos, and she has a delicate silver bracelet on one wrist. She stands confidently, exuding elegance and intrigue.",
            image_prompt_short="Neo-Noir Color Pop style, full-body of an elegant 20s femme fatale in black dress, red lips, and confident pose."
        ),
        Character(
            nickname="killer",
            name="The Killer, Unknown",
            role="Antagonist and orchestrator of the murders",
            gender="Ambiguous",
            race="Human",
            age="Unknown",
            description="A shadowy figure hidden within the city's high society, pulling strings from behind the scenes.",
            personality="Cold, calculating, with a ruthless approach to covering up secrets.",
            physical_appearance="Usually hidden, but occasionally glimpsed as a silhouette in a dark suit with gloves.",
            catch_phrase="Some secrets die with their keepers.",
            animation_description="The killer moves silently, blending into shadows and avoiding direct light. Their steps are cautious and calculated, as if always planning their next move.",
            voice_description="Low and menacing, with a tone that chills the listener.",
            internal_conflict="Driven by a need to maintain control and keep dark secrets hidden at any cost.",
            character_arc=CharacterArc(
                initial_state="Confident in their ability to evade detection.",
                final_state="Desperate and reckless as Jack gets closer to the truth.",
                key_moments=[
                    "Leaves cryptic messages at crime scenes.",
                    "Observes Jack from a distance, staying just out of reach.",
                    "Realizes their control is slipping as Jack uncovers their identity."
                ]
            ),
            image_prompt="Neo-Noir Color Pop style, full-body shot of a shadowy, ambiguous figure mostly hidden by darkness. They wear a dark suit with a high-collared coat that obscures the face, creating a silhouette effect. Their gloves are black leather, and the hint of a dark fedora is barely visible over the collar, adding to the ominous, anonymous vibe. Only the outline of their face is visible under the hat, giving an impression of a cold, calculating presence. Their stance is cautious and controlled, exuding an air of menace and secrecy.",
            image_prompt_short="Neo-Noir Color Pop style, full-body of an ominous figure in dark suit, gloves, and hat, face obscured."
        )
    ],
    acts=[
        Act(
            act_id="shadows_secrets",
            title="Act 1 - Shadows and Secrets",
            scenes=[
                Scene(
                    scene_id="murder_alley",
                    title="Murder in the Alley",
                    description="Jack is called to investigate a murder in a narrow alley, finding the first cryptic clue left by the killer.",
                    setting="A dimly lit alleyway with brick walls covered in grime and faded posters.",
                    key_actions=[
                        "Jack examines the scene, noting a strange mark left near the body.",
                        "Discovers a small red rose pinned to the victim's coat.",
                        "Reflects on the eerie feeling that he's being watched from the shadows."
                    ],
                    characters_involved=["jack"],
                    location="Alleyway",
                    time_of_day="Night",
                    lighting="Low, with a flickering streetlight casting dim light across the scene",
                    mood="Dark and foreboding",
                    background_animation="Slow tracking shot, medium-wide angle, with soft, flickering streetlight casting an eerie glow. Subject: A dimly lit 1950s alley with brick walls. Damp, grimy bricks covered in faded posters, some torn and peeling. Scene: Mist drifts lazily through the alley as occasional footsteps echo faintly. Atmosphere: Tense and foreboding.",
                    background_image_prompt="Neo-Noir Color Pop style of a gritty 1950s alley, dimly lit by a flickering streetlight that creates elongated shadows along damp brick walls. The ground is strewn with trash and faint mist drifting through, enhancing the mysterious, noir atmosphere. The only color contrast is a small red rose pinned to a victim's coat.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A dimly lit 1950s alley with damp brick walls, trash scattered on the ground, and faded posters peeling off the walls. A single, flickering streetlight casts a warm, yellowish glow that adds contrast between shadows and light.
Layer 2 (Middle ground): In the midst of the alley, a crime scene with a body is mostly obscured in shadow. The mist swirls around, adding an atmospheric, smoky quality to the air.
Layer 3 (Foreground): A tall man with a rugged appearance, in a trench coat and fedora, crouches near the body with a serious expression. A small red rose pinned to the victim's coat stands out as the only color contrast in the grayscale setting.
Atmosphere: The scene is tense, shadowed, and filled with mystery, as the detective studies the crime scene under the dim streetlight.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a gritty 1950s alley with a detective examining a murder scene under a flickering streetlight. A red rose stands out as the only color against the grayscale setting."

                ),
                Scene(
                    scene_id="jazz_club",
                    title="Secrets in the Jazz Club",
                    description="Jack follows a lead to a smoky jazz club, where he encounters Evelyn performing on stage.",
                    setting="Dimly lit jazz club filled with smoky air and low lighting.",
                    key_actions=[
                        "Jack observes Evelyn's performance, captivated by her allure.",
                        "After the show, he approaches her with questions, probing her knowledge of the murders.",
                        "Evelyn gives subtle hints, leaving Jack both intrigued and suspicious."
                    ],
                    characters_involved=["jack", "evelyn"],
                    location="Jazz club",
                    time_of_day="Night",
                    lighting="Low, with a spotlight on stage casting a warm glow on Evelyn.",
                    mood="Tense and intimate",
                    background_animation="Slow pan, medium shot, under warm, dim lighting. Subject: A smoky jazz club interior with patrons seated at small tables. Scene: Low lights with a warm spotlight on the stage, where a woman in a red dress performs; light smoke drifts through the scene. Atmosphere: Intimate, tense, and mysterious.",
                    background_image_prompt="Neo-Noir Color Pop style of a smoky 1950s jazz club with dim table lamps scattered around. Patrons sit at small round tables, watching a warm spotlight on the stage, highlighting the main performer in an elegant red dress against the grayscale background.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): The dim interior of a smoky 1950s jazz club. Small tables with dim lamps and smoke drifting through create a moody, intimate ambiance.
Layer 2 (Middle ground): A jazz band plays in shadows, their instruments silhouetted in grayscale. A warm spotlight highlights the performer, casting her silhouette in sharp contrast.
Layer 3 (Foreground): Center stage stands a tall woman in a striking red dress, under the spotlight. Her movements are fluid, and her expression is confident yet enigmatic. Across from her, a rugged man in a trench coat watches, sipping his drink with a focused gaze.
Atmosphere: Smoky, tense, and alluring, with the red dress as the only color pop in the grayscale setting. The air feels thick with secrets and intrigue as the detective studies her from afar.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a smoky jazz club, where a woman in a red dress performs under a warm spotlight. A detective observes her from the shadows, adding mystery to the grayscale scene."

                ),
                Scene(
                    scene_id="mysterious_witness",
                    title="The Mysterious Witness",
                    description="Jack receives an anonymous call from someone claiming to have seen the killer.",
                    setting="Jack's dark, cluttered office.",
                    key_actions=[
                        "Jack takes the call and listens intently to the whispered voice.",
                        "The caller hints at a connection to high society and warns Jack to be careful.",
                        "The line suddenly cuts, leaving Jack tense and filled with questions."
                    ],
                    characters_involved=["jack"],
                    location="Jack's office",
                    time_of_day="Late evening",
                    lighting="Dim, lit by a single desk lamp.",
                    mood="Suspenseful and foreboding",
                    background_animation="Slow zoom-in, close-up shot, under low, warm lighting from a desk lamp. Subject: A dimly lit office desk with scattered papers and case files. Scene: The shadows stretch across the floor and walls, giving the room an atmosphere of suspense and secrecy. Atmosphere: Suspenseful and isolating.",
                    background_image_prompt="Neo-Noir Color Pop style of a detective's dim, cluttered office at night, with a desk lamp casting warm light on scattered papers and half-opened case files. Shadows stretch across the floor and walls, adding tension to the suspenseful scene.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A dimly lit, 1950s detective's office with shadows stretching across the walls. Papers, folders, and empty coffee cups lie scattered on the wooden desk.
Layer 2 (Middle ground): A warm desk lamp casts focused light on a few open files, casting shadows that add a heavy atmosphere to the room.
Layer 3 (Foreground): A worn, empty chair sits at the desk, as if recently vacated by someone in a rush. The phone sits on the edge, with the receiver slightly askew, adding to the air of suspense.
Atmosphere: Suspenseful, foreboding, and reflective, with long shadows adding tension and a feeling of isolation.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a detective's dim, cluttered office at night, lit by a single desk lamp casting shadows over scattered papers."

                )
            ]
        ),
        Act(
            act_id="dangerous_liaisons",
            title="Act 2 - Dangerous Liaisons",
            scenes=[
                Scene(
                    scene_id="the_ball",
                    title="The Ball in High Society",
                    description="Jack attends an elite ball hosted by a powerful family, suspecting their involvement.",
                    setting="A grand ballroom with crystal chandeliers and elegantly dressed guests.",
                    key_actions=[
                        "Jack arrives at the ball, blending in with the crowd to observe.",
                        "He spots Evelyn and confronts her, pressing her for answers.",
                        "A mysterious man approaches, exchanging a tense look with Jack before disappearing into the crowd."
                    ],
                    characters_involved=["jack", "evelyn"],
                    location="Ballroom",
                    time_of_day="Evening",
                    lighting="Warm, with a golden glow from chandeliers.",
                    mood="Elegant yet tense",
                    background_animation="Slow crane shot, wide-angle with warm, golden lighting from chandeliers. Subject: A grand ballroom with opulently dressed guests and elegant crystal chandeliers. Subject Movement: Guests chatting, dancing, and mingling while subtle tension builds in the background. Scene: Ornate decorations, rich drapery, and polished floors highlight the extravagance. Atmosphere: Luxurious yet subtly tense, as hidden intentions linger.",
                    background_image_prompt="Neo-Noir Color Pop style of a grand ballroom illuminated by golden chandelier lights. Elegantly dressed guests fill the room, their conversations and movements creating a lively yet tense atmosphere. The rich drapery and polished floors add to the luxurious feel.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A grand ballroom with crystal chandeliers casting a warm golden glow, illuminating rich drapery and polished floors.
Layer 2 (Middle ground): Elegantly dressed guests fill the room, chatting and mingling, adding layers of movement to the scene.
Layer 3 (Foreground): A tall man with rugged features, in a dark suit, stands tensely as he observes a poised woman in a red dress nearby, her confident stance and piercing gaze adding to the mystery.
Atmosphere: Luxurious, refined, with a tense undercurrent, as hidden motives begin to surface amid the glamour.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of an elegant ballroom with a tense atmosphere, where a detective observes a woman in a red dress among the wealthy guests."

                ),
                Scene(
                    scene_id="alley_confrontation",
                    title="Cornered in the Alley",
                    description="On his way home, Jack is ambushed by two men in an alley who warn him to stop investigating.",
                    setting="Dimly lit alley with brick walls and scattered trash.",
                    key_actions=[
                        "The men threaten Jack, telling him to 'forget the case'.",
                        "Jack fights back, using his street smarts to fend off the attackers.",
                        "After the scuffle, he realizes the stakes are higher than he thought."
                    ],
                    characters_involved=["jack"],
                    location="Alleyway",
                    time_of_day="Night",
                    lighting="Flickering streetlight casting shadows.",
                    mood="Tense and gritty",
                    background_animation="Handheld, medium-close shot under flickering streetlight. Subject: Dark alley with brick walls and trash. Subject Movement: Figures emerge from the shadows, circling their target in a threatening manner. Scene: Scattered trash and grime add to the rough environment. Atmosphere: Tense and gritty, heightening the sense of danger.",
                    background_image_prompt="Neo-Noir Color Pop style of a dark alley with flickering streetlight casting harsh shadows. Trash litters the ground, and the grimy brick walls add to the grittiness as two shadowy figures close in.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A dim, flickering streetlight casts shadows on grimy brick walls, intensifying the darkness and rough atmosphere.
Layer 2 (Middle ground): Two shadowy figures emerge, partially hidden in the shadows, moving slowly toward their target.
Layer 3 (Foreground): A man with rugged features, trench coat torn from the struggle, stands with a determined stance, fists clenched and ready to defend himself.
Atmosphere: Gritty, intense, and foreboding, capturing the danger and suspense in the narrow, dark alley.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a gritty alley with flickering light, where a detective faces two shadowy figures moving toward him."
                ),
                Scene(
                    scene_id="evelyn_confession",
                    title="Evelyn's Confession",
                    description="Evelyn shows up at Jack's apartment, revealing her fears and connection to the case.",
                    setting="Jack's sparsely furnished apartment.",
                    key_actions=[
                        "Evelyn confides in Jack, revealing her ties to high society.",
                        "She hints at knowing who the killer is, but fears for her life.",
                        "They share a quiet moment of trust, deepening their connection."
                    ],
                    characters_involved=["jack", "evelyn"],
                    location="Jack's apartment",
                    time_of_day="Night",
                    lighting="Low, with soft light from a nearby streetlamp.",
                    mood="Intimate and tense",
                    background_animation="Close-up, handheld shot with soft, low light from a nearby streetlamp. Subject: A sparsely furnished 1950s apartment. Subject Movement: A woman and man sit across from each other, exchanging glances as they converse quietly. Scene: Worn furniture and empty shelves add to the stark atmosphere. Atmosphere: Intimate yet tense, as vulnerability and trust mix with fear.",
                    background_image_prompt="Neo-Noir Color Pop style of a dimly lit, sparse 1950s apartment. The soft light from a nearby streetlamp filters through, casting shadows over worn furniture as two figures sit closely, deep in conversation.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A sparsely furnished 1950s apartment with worn furniture and bare walls, illuminated by soft light from a nearby streetlamp.
Layer 2 (Middle ground): The dim light casts gentle shadows, adding an intimate but tense feel to the room.
Layer 3 (Foreground): A tall, rugged man in a trench coat sits opposite a poised woman in a sleek black dress, her expression vulnerable yet guarded. Their close proximity reflects a sense of trust tempered with unease.
Atmosphere: Intimate, shadowed, and filled with tension, capturing a rare moment of openness in a setting clouded by mistrust and fear.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a dim, sparsely furnished apartment, where two figures sit in tense conversation, illuminated by a soft streetlight."

                )
            ]
        ),
        Act(
            act_id="the_killers_web",
            title="Act 3 - The Killer's Web",
            scenes=[
                Scene(
                    scene_id="secret_hideout",
                    title="The Secret Hideout",
                    description="Jack discovers a hidden office with files detailing each victim and their ties to high society.",
                    setting="An abandoned office with dusty furniture and scattered papers.",
                    key_actions=[
                        "Jack finds documents linking the victims to a powerful family.",
                        "He realizes the murders are part of a larger conspiracy.",
                        "A photograph falls from a file, showing Evelyn with the killer."
                    ],
                    characters_involved=["jack"],
                    location="Abandoned office",
                    time_of_day="Evening",
                    lighting="Dim, with shadows adding to the mystery.",
                    mood="Revelatory and foreboding",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A dusty, abandoned office with broken blinds, casting thin slivers of light that reveal a neglected atmosphere.
Layer 2 (Middle ground): Scattered papers and files lie strewn across the floor and desk, some slightly yellowed with age. Shadows add a sense of mystery and concealment.
Layer 3 (Foreground): A rugged man with a trench coat, tense and focused, examines a document closely. His face is partially illuminated, creating an expression of revelation and intrigue.
Atmosphere: Dark and suspenseful, capturing the secrecy and hidden truths of the room.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a dusty, abandoned office where a detective examines documents under dim light."

                ),
                Scene(
                    scene_id="ballroom_betrayal",
                    title="Betrayal in the Ballroom",
                    description="Jack confronts Evelyn, suspecting her complicity, only to be ambushed by the killer.",
                    setting="An opulent ballroom, now empty and shadowed.",
                    key_actions=[
                        "Jack questions Evelyn, pressing her about the photograph.",
                        "Before she can respond, the killer appears, catching Jack off guard.",
                        "Evelyn tries to protect Jack, revealing her true allegiance."
                    ],
                    characters_involved=["jack", "evelyn", "killer"],
                    location="Ballroom",
                    time_of_day="Night",
                    lighting="Low, with dramatic shadows cast by chandeliers.",
                    mood="Suspenseful and intense",
                    background_animation="Slow zoom-in, low-angle, with hard, direct lighting casting dramatic chandelier shadows. Subject: An opulent, empty ballroom, shadows stretching across the polished floors. Subject Movement: A figure steps cautiously from the shadows, moving silently toward the center. Scene: Ornate decor and hanging chandeliers now cloaked in shadow. Atmosphere: Suspenseful and intense, as danger lurks in the dark corners of elegance.",
                    background_image_prompt="Neo-Noir Color Pop style of an opulent, shadow-filled ballroom with chandeliers casting dramatic shadows. Rich decor is cloaked in darkness, adding a sense of unease as a lone figure steps forward.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): An opulent, empty ballroom draped in shadows, with crystal chandeliers casting dramatic, intricate patterns across the polished floor.
Layer 2 (Middle ground): Rich, dark decor, including ornate columns and drapery, adds a sense of elegance clouded by hidden motives.
Layer 3 (Foreground): A rugged man in a trench coat stands tense, confronting a woman in a red dress nearby, her expression conflicted. A shadowy figure lurks, ready to strike.
Atmosphere: Elegant yet suspenseful, with tension building as hidden intentions come to light.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a shadowed ballroom, where a tense detective confronts a woman in red as danger lurks nearby."

                ),
                Scene(
                    scene_id="rooftop_showdown",
                    title="Showdown on the Rooftop",
                    description="Jack and the killer face off in a final confrontation on the rooftop.",
                    setting="Rooftop with a neon-lit cityscape.",
                    key_actions=[
                        "The killer taunts Jack, revealing twisted motives for the murders.",
                        "Jack confronts his own fears and past as he wrestles with the killer.",
                        "After a tense struggle, Jack gains the upper hand and captures the killer."
                    ],
                    characters_involved=["jack", "killer"],
                    location="Rooftop",
                    time_of_day="Late night",
                    lighting="Neon lights casting an eerie glow.",
                    mood="Climactic and intense",
                    background_animation="Handheld, close-up, with sweeping pan, illuminated by intense neon lighting from the skyline. Subject: A dim rooftop with city lights flickering in the distance. Subject Movement: Two figures grapple, each maneuvering against the other. Scene: Rooftop ledge overlooking a neon-lit cityscape. Atmosphere: Climax and intensity, with emotions and tension reaching a peak amid the glowing, chaotic city below.",
                    background_image_prompt="Neo-Noir Color Pop style of a gritty rooftop under neon-lit skies. The city lights cast an eerie glow, illuminating the fierce struggle between two silhouetted figures near the edge.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): Neon-lit cityscape, with tall buildings casting fragmented, colorful lights against a moody night sky.
Layer 2 (Middle ground): The rooftop edge shows a worn, gravel-covered surface, with faint reflections of the neon lights.
Layer 3 (Foreground): Two figures, locked in a tense struggle, their expressions intense as they grapple near the ledge. The man's rugged face shows determination, while the shadowed figure exudes menace.
Atmosphere: Climactic and intense, as the fight escalates under the neon-lit night, bringing the final confrontation to life.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a rooftop under neon lights, where two figures engage in a fierce struggle near the edge."

                )
            ]
        ),
        Act(
            act_id="final_revelation",
            title="Act 4 - The Final Revelation",
            scenes=[
                Scene(
                    scene_id="unmasking_truth",
                    title="Unmasking the Truth",
                    description="Jack learns that the killer was manipulated by powerful figures in high society.",
                    setting="Police interrogation room.",
                    key_actions=[
                        "Jack questions the killer, uncovering the role of high-society figures in the murders.",
                        "The killer admits to acting under pressure from influential people.",
                        "Jack realizes justice may be out of reach as powerful figures evade responsibility."
                    ],
                    characters_involved=["jack", "killer"],
                    location="Interrogation room",
                    time_of_day="Morning",
                    lighting="Harsh fluorescent lighting.",
                    mood="Reflective and somber",
                    background_animation="Static shot, medium close-up with harsh, direct fluorescent lighting. Subject: A stark, sterile interrogation room with a metal table and two chairs. Subject Movement: The figure on the opposite side leans forward and back subtly, reflecting emotional strain. Scene: Cold, impersonal space with flat, unyielding light that emphasizes the tense atmosphere. Atmosphere: Reflective, heavy, and solemn, underscoring the weight of exposed secrets.",
                    background_image_prompt="Neo-Noir Color Pop style of a stark, impersonal interrogation room with metal furniture and harsh fluorescent lights. The room is bare, amplifying the sense of isolation as two figures face each other in tense conversation.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): A cold, sterile interrogation room with metal walls and fluorescent lighting casting stark, unflattering light.
Layer 2 (Middle ground): A plain metal table stands between two chairs, with shadows adding to the sense of isolation and coldness.
Layer 3 (Foreground): A rugged man with a weary expression sits opposite a shadowed figure, leaning in with a look of frustration and realization, emphasizing his internal conflict as he questions the figure across from him.
Atmosphere: Reflective and somber, with the sterile setting amplifying the emotional weight of exposed secrets.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of a cold interrogation room with two figures in tense conversation under harsh fluorescent light."

                ),
                Scene(
                    scene_id="farewell_with_evelyn",
                    title="A Farewell with Evelyn",
                    description="Jack meets Evelyn one last time, acknowledging their complex relationship.",
                    setting="Empty jazz club in the early morning light.",
                    key_actions=[
                        "Jack and Evelyn share a bittersweet moment, acknowledging the darkness they both endured.",
                        "Evelyn leaves the city, hoping to start fresh, while Jack reflects on his journey.",
                        "Jack watches her walk away, knowing his work continues in a city filled with shadows."
                    ],
                    characters_involved=["jack", "evelyn"],
                    location="Jazz club",
                    time_of_day="Morning",
                    lighting="Soft, with beams of morning sunlight filtering through the windows.",
                    mood="Bittersweet and reflective",
                    background_animation="Slow zoom-out, wide shot with soft, natural morning light streaming through windows. Subject: A dimly lit, empty jazz club with scattered chairs and tables. Subject Movement: A woman walks slowly towards the door, her silhouette framed by the sunlight. Scene: Dust particles drift lazily in the soft morning light, creating an atmosphere of quiet and melancholy. Atmosphere: Bittersweet and reflective, symbolizing closure and the beginning of a new chapter.",
                    background_image_prompt="Neo-Noir Color Pop style of an empty jazz club bathed in soft morning light, with beams of sunlight filtering through the dusty air. The club feels quiet and nostalgic, as a lone figure walks slowly toward the exit.",
                    scene_image_prompt="""Neo-Noir Color Pop style:
Layer 1 (Background): Dim, empty jazz club with chairs and tables cast in soft morning light filtering through large windows, creating a nostalgic atmosphere.
Layer 2 (Middle ground): Dust particles catch the light, floating gently in the air, adding to the sense of stillness.
Layer 3 (Foreground): A woman in an elegant red dress walks slowly toward the door, her figure silhouetted by the warm, diffused light. Nearby, a rugged man watches her departure with a reflective gaze.
Atmosphere: Bittersweet and reflective, capturing the moment of closure and a quiet goodbye amidst the lingering shadows.""",
                    scene_image_prompt_short="Neo-Noir Color Pop style of an empty jazz club in soft morning light, with a lone woman walking towards the exit and a man watching her departure, capturing a bittersweet farewell."

                )
            ]
        )
    ],

    story_beats=[
        StoryBeat(name="First Murder", description="Jack finds the first cryptic clue at the scene.", scene="Murder in the Alley"),
        StoryBeat(name="Encounter with Evelyn", description="Jack meets Evelyn, who may have information about the murders.", scene="Secrets in the Jazz Club"),
        StoryBeat(name="The Ambush", description="Jack is threatened by thugs warning him off the case.", scene="Cornered in the Alley"),
        StoryBeat(name="Evelyn's Confession", description="Evelyn reveals her connection to the case.", scene="Evelyn's Confession"),
        StoryBeat(name="Uncovering the Plot", description="Jack finds evidence of a conspiracy in high society.", scene="The Secret Hideout"),
        StoryBeat(name="Final Confrontation", description="Jack and the killer face off on the rooftop.", scene="Showdown on the Rooftop"),
        StoryBeat(name="Resolution", description="Jack uncovers the truth but realizes justice is incomplete.", scene="Unmasking the Truth")
    ],
    subplots=[
        Subplot(title="Jack's Haunted Past", description="Jack confronts memories of an old, unsolved case.", related_characters=["Jack Holden, the Detective"]),
        Subplot(title="Evelyn's Secrets", description="Evelyn's connection to the murders unfolds, raising questions about her motives.", related_characters=["Evelyn Blackwood, the Femme Fatale"])
    ],
    emotional_arc=[
        EmotionalArc(stage="Determination", description="Jack is resolute in his pursuit of justice."),
        EmotionalArc(stage="Doubt", description="As evidence mounts, Jack begins to question whom he can trust."),
        EmotionalArc(stage="Reflection", description="Jack faces his own inner demons and unresolved trauma.")
    ],

    conflict_type="Man vs. Man and Man vs. Self",
    themes=["Truth and Deception", "The Cost of Secrets"],
    motifs=["Dark Alleys and Shadows", "Red Rose as a Clue"]
)

few_shot_stories = [
    few_shot_story_1,
]

In [ ]:
# Few-shot outputs
from utils import deindent

few_shot_book_scene_1 = deindent("""
    Detective Jack Holden pulled the collar of his trench coat tight against his neck, stepping cautiously into the alley. The faint glow of a flickering streetlight cast long, wavering shadows across the brick walls, where faded posters peeled and clung in stubborn defiance of the damp air. His polished but worn leather shoes crunched over the scattered litter, breaking the silence of the deserted street.

    At the end of the alley lay the body—a man sprawled in a dark overcoat, face pale and still under the dim light. Jack's eyes, sharp and unyielding, swept over the scene. Another victim. Another dead end.

    He crouched, letting his gaze linger on the details. A dark stain soaked the ground beneath the body, nearly black in the low light. The face of the victim, frozen in shock, seemed to tell Jack all he needed to know about the fear that had gripped him in his final moments. But it wasn't the fear that caught Jack's attention. It was the rose.

    Pinned neatly to the victim's lapel, a single red rose glistened, its color standing out against the drab, grayscale world around it. Jack reached out, brushing the petals lightly with his fingertips. It was real—fresh, vibrant, placed with meticulous care. The same mark, the same calling card he'd seen before.

    “Another one…” he muttered to himself, his voice barely a whisper, lost in the vast silence of the alley.

    The breeze shifted, carrying with it a faint scent of smoke from the city's night air, and Jack felt a chill crawl up his spine. He glanced over his shoulder, half-expecting to see someone lurking in the shadows, watching. But the alley was empty—save for him and the dead.

    Straightening up, he cast one more look at the scene. He had seen plenty of murders in his career, but this one felt different. There was something calculated here, something personal. The rose wasn't just a flourish; it was a message. A warning, perhaps, to those who dared come too close.

    Jack's fingers brushed the scar on his cheek, a faint reminder of a case he'd never solved, a mistake that haunted him every night. “Who are you?” he murmured, looking down at the rose. “And why roses?”

    The silence of the alley offered no answers. Just shadows, shifting in the dim light.

    As Jack turned to leave, he couldn't shake the feeling that eyes were on him, watching his every move. The city held its secrets close, and tonight, Jack felt closer to the heart of those secrets than ever before.

    But the night was young, and so was his investigation.
""")

few_shot_outputs = [
    few_shot_book_scene_1,
]

In [ ]:
from model import Story, Act, Scene, Character

def generate_story_overview(story: Story) -> str:
    """Generates a Markdown overview for the story, including title, genre, setting, themes, and other high-level details."""
    markdown = f"# {story.title}\n\n"
    markdown += f"**Medium:** {story.medium}\n\n"
    markdown += f"**Genre:** {story.genre}\n\n"
    markdown += f"**Visual Style:** {story.visual_style}\n\n"
    markdown += f"**Time Period:** {story.time_period}\n\n"
    markdown += f"**Location:** {story.location}\n\n"
    markdown += f"**Narrative Perspective:** {story.narrative_perspective}\n\n"
    markdown += f"**Conflict Type:** {story.conflict_type}\n\n"
    
    # Themes and motifs
    if story.themes:
        markdown += f"**Themes:** {', '.join(story.themes)}\n\n"
    if story.motifs:
        markdown += f"**Motifs:** {', '.join(story.motifs)}\n\n"

    # Story Beats
    if story.story_beats:
        markdown += "## Story Beats\n\n"
        for beat in story.story_beats:
            markdown += f"- **{beat.name}:** {beat.description} (Scene: {beat.scene})\n"
        markdown += "\n"

    # Subplots
    if story.subplots:
        markdown += "## Subplots\n\n"
        for subplot in story.subplots:
            markdown += f"### Subplot: \"{subplot.title}\"\n"
            markdown += f"#### Description:\n{subplot.description}\n\n"
            markdown += f"#### Related Characters (nicknames):\n{', '.join(subplot.related_characters)}\n\n"
        markdown += "\n"

    # Emotional Arc
    if story.emotional_arc:
        markdown += "## Emotional Arc\n\n"
        for arc in story.emotional_arc:
            markdown += f"- **{arc.stage}:** {arc.description}\n"
        markdown += "\n"

    markdown += f"## Original User Prompt:\n{story.prompt}\n\n"
    markdown += f"## Plot Overview:\n{story.plot_overview}\n\n"
    return markdown

def generate_character_summary(character: Character) -> str:
    """Generates a Markdown summary for a single character, including role, description, and internal conflict."""
    markdown = f"### Character: \"{character.name}\"\n\n"
    markdown += f"#### Role:\n{character.role}\n\n"
    markdown += f"#### Description:\n{character.description}\n\n"
    markdown += f"#### Personality:\n{character.personality}\n\n"
    markdown += f"#### Appearance:\n{character.physical_appearance}\n\n"
    markdown += f"#### Internal Conflict:\n{character.internal_conflict or "none"}\n\n"
    if character.character_arc:
        markdown += f"#### Character Arc:\n"
        markdown += f"- Initial State: {character.character_arc.initial_state}\n"
        markdown += f"- Final State: {character.character_arc.final_state}\n"
        markdown += f"- Key Moments: {', '.join(character.character_arc.key_moments)}\n\n"
    return markdown

def generate_act_summary(act: Act, include_scenes: bool = True) -> str:
    """Generates a Markdown summary for an act, optionally including details on each scene."""
    markdown = f'### Act: "{act.title}"\n\n'
    markdown += f"**Description:**\n{act.description}\n\n"

    if include_scenes:
        for scene in act.scenes:
            markdown += generate_scene_summary(scene)
    return markdown

def generate_scene_summary(scene: Scene) -> str:
    """Generates a Markdown summary for a single scene, including setting, characters involved, and key actions."""
    markdown = f"#### Scene: \"{scene.title}\"\n\n"
    markdown += f"**Scene ID:** {scene.scene_id}\n"
    markdown += f"**Setting:** {scene.setting}\n"
    markdown += f"**Time of Day:** {scene.time_of_day}\n"
    markdown += f"**Location:** {scene.location}\n"
    markdown += f"**Lighting:** {scene.lighting}\n"
    markdown += f"**Mood:** {scene.mood}\n"
    markdown += f"**Characters Involved (by nickname):** {', '.join(scene.characters_involved)}\n"
    markdown += f"**Props:** {', '.join(scene.props)}\n"

    if scene.key_actions:
        markdown += f"##### Key Actions:\n"
        for action in scene.key_actions:
            markdown += f"- {action}\n"
        markdown += "\n"

    markdown += f"##### Description:\n{scene.description}\n\n"

    return markdown

def generate_full_story_summary(story: Story, include_characters: bool = True, include_acts: bool = True, include_scenes: bool = True) -> str:
    """Generates a full Markdown summary for the entire story, including overview, characters, acts, and scenes."""
    markdown = generate_story_overview(story)
    
    # Characters
    if include_characters:
        markdown += "\n\n---\n\n## Characters\n\n"
        for character in story.characters:
            markdown += generate_character_summary(character) + "\n"
    
    # Acts and Scenes
    if include_acts:
        markdown += "\n\n---\n\n## Acts and Scenes\n\n"
        for act in story.acts:
            markdown += generate_act_summary(act, include_scenes=include_scenes) + "\n"

    return markdown


In [ ]:
print(generate_full_story_summary(few_shot_story_1))
# display(Markdown(generate_story_summary_markdown(few_shot_story_1)))

# The Shadows of Midnight

**Medium:** Film

**Genre:** Crime Mystery

**Visual Style:** Neo-Noir Color Pop

**Time Period:** 1950s

**Location:** A gritty urban cityscape with smoky bars, dim alleyways, and grand ballrooms

**Narrative Perspective:** Third-person limited, primarily following Jack Holden's viewpoint

**Conflict Type:** Man vs. Man and Man vs. Self

**Themes:** Truth and Deception, The Cost of Secrets

**Motifs:** Dark Alleys and Shadows, Red Rose as a Clue

## Story Beats

- **First Murder:** Jack finds the first cryptic clue at the scene. (Scene: Murder in the Alley)
- **Encounter with Evelyn:** Jack meets Evelyn, who may have information about the murders. (Scene: Secrets in the Jazz Club)
- **The Ambush:** Jack is threatened by thugs warning him off the case. (Scene: Cornered in the Alley)
- **Evelyn's Confession:** Evelyn reveals her connection to the case. (Scene: Evelyn's Confession)
- **Uncovering the Plot:** Jack finds evidence of a conspiracy in high society. 

In [ ]:
# def stream_llm_response(response, progress=None):
#     line_len = 0
#     for r in response:
#         if progress:
#             progress.value += 1
#         print(r.delta, end="")
#         line_len += len(r.delta)
#         if line_len > 120:
#             print()
#             line_len = 0
#         elif "\n" in r.delta:
#             line_len = 0
#     return r.message.content

In [ ]:
from IPython.display import Markdown, display
import ipywidgets as widgets
from model_text import display_messages
from llama_index.core.llms import ChatMessage
from model_text import llm, stream_llm_response
from utils import blank_story, blank_story_dialog, deindent
from model import Story, DialogueLine, ActDialogue, SceneDialogue, StoryDialogue, ActDialogue, SceneDialogue, DialogueLine

# Define exclusions for the user prompt and LLM response
request_excludes = {
    "characters": {
        "__all__": {
            "animation_description": True,
            "voice_description": True,
            # "internal_conflict": True,
            # "character_arc": True,
            "voice_sample": True,
            "catch_phrase": True,
            "props": True,
            "image_prompt": True,
            "image_prompt_short": True,
        }
    }, 
    "acts": {
        "__all__": {
            "scenes": {
                "__all__": {
                    "scene_image_prompt": True,
                    "scene_image_prompt_short": True,
                    "background_animation": True,
                    "background_image_prompt": True,
                    "dialogue": True,
                }
            }
        }
    },
}

def generate_manuscript(story: Story) -> StoryDialogue:
    total_scenes = sum([len(act.scenes) for act in story.acts])
    approx_chunks = 500 # Just to show progress as a chunks come in
    progress = widgets.IntProgress(value=0, min=0, max=total_scenes * approx_chunks)
    scene_number = 0
    display(progress)

    # Initialize StoryDialogue
    story_dialogue = story.get_story_dialogue()
    if not story_dialogue:
        story_dialogue = StoryDialogue()
    story.set_story_dialogue(story_dialogue)

    story_context = generate_full_story_summary(story)

    # TODO: Support for narrator/dialogue distinction

    display(Markdown(f"### Generating Story: {story.title}"))

    for act in story.acts:
        display(Markdown(f"#### Generating Act: {act.title}"))
        act_dialogue = ActDialogue(act_id=act.act_id)
        story_dialogue.act_dialogues.append(act_dialogue)

        for scene in act.scenes:
            progress.value = scene_number * approx_chunks
            scene_number += 1
        
            display(Markdown(f"##### Generating Scene: {scene.title}"))

            scene_dialogue = SceneDialogue(scene_id=scene.scene_id)
            act_dialogue.scene_dialogues.append(scene_dialogue)

            system_message = deindent(f"""
                You are an exceptionally talented and creative book author, specializing in crafting vivid, narrative-driven chapters for a {story.medium}.
                
                Write a complete chapter in natural, printable prose based on the scene provided. The output should be **only the narrative content of the chapter** as if it were ready for publication. 

                Guidelines:
                - No headers, titles, or meta information: Do not add any headers, titles, genre notes, or extra explanations.
                - Only narrative prose: Write as if this is a final printed chapter in a novel, without character labels or any structural cues.
                - Make it immersive: Use rich descriptions of settings, character actions, thoughts, and dialogue to move through the scene, and focus on sensory details to make the reader feel present.
                - Pacing: Fully develop each part of the scene, including character interactions, conflicts, emotions, and subtleties. Take your time with the narrative, allowing it to unfold naturally and build tension or depth.

                Suggested Structure:
                - **Begin** with an immersive setting description to establish time, place, and atmosphere.
                - **Progress** through the scene using detailed character actions, interactions, thoughts, and emotions, taking the time to explore each element.
                - **Conclude** naturally, allowing for smooth continuity into the following chapter if applicable.

                Do not output anything but a completed chapter in markdown format.
                Only output ASCII characters. Do not include any special characters or emojis like smark quotes and em dashes.

                DO NOT OUTPUT ANYTHING BUT A COMPLETED CHAPTER!

                Output in markdown. Do not include chapter titles.

                Here is the overview of the story for context: 
            """)
            # TODO: adjust the chapter length according to the medium
            system_message += "\n\n" + story_context
            # print(system_message)
            messages = [ChatMessage(role="system", content=system_message)]
            user_prompt = deindent(f"""
                Write an expanded, full-length chapter for the scene "{scene.title}" in the act "{act.title}" in markdown format. 
                Focus on creating a fully immersive experience, as though it is a complete chapter in a novel. Do not include the title or any meta information.
            """)

            messages.append(ChatMessage(role="user", content=user_prompt))
            
            # Debug display of messages
            display_messages(messages)

            # Generate dialogue using llm_json
            # response = llm.chat(messages, max_new_tokens=100000).message.content
            response = llm.stream_chat(messages, max_new_tokens=100000)
            scene_dialogue.content = stream_llm_response(response, progress=progress)

            # display(Markdown(f"### Scene Content: {scene_dialogue.content}"))

            # if settings.DEBUG:
            # display(Markdown(f"```\n{response}\n```"))

            story.save_to_directory(settings.STORY_DIR + "/step_14")
            display(Markdown(f":white_check_mark: Saved story progress!"))


    return story_dialogue


## Generate the book manuscript

In [ ]:
manuscript = generate_manuscript(story)

IntProgress(value=0, max=3000)

### Generating Story: Beyond the Pixel Veil

#### Generating Act: Act 1: Beyond the Screen

##### Generating Scene: The Underdog's Rise

The air was electric with anticipation as Meera 'Midnight' Singh stepped onto the vibrant, neon-lit stage of New Mumbai's
 Cyber Arena. The crowd's deafening roar enveloped her, a cacophony of cheers and chants that threatened to consume her entire
 being. With a deep breath, she slipped on her trusty headset, the familiar weight a comforting reminder of the battles she
 was about to face.

As she gazed out into the sea of expectant faces, Meera's thoughts drifted back to the countless hours spent honing her craft
 in cramped, dimly lit internet cafes. The underdog from the outskirts of New Mumbai had finally made it to the big leagues
 – the Eon Gaming Tournament's grand finale. Her team, 'Midnight's Marauders,' was set to take on the reigning champions,
 'Specter's Squad,' led by the charismatic Jax 'Specter' Lee.

Meera's eyes locked onto her teammates, each one a vital cog in their well-oiled machine. There was Rohan 'Brawler' Jensen
, their tanky powerhouse; Leela 'Lumina' Rao, th

...

The darkness was absolute, punctuated only by the faint hum of emergency generators and the soft murmur of hushed conversations
. Meera's eyes, accustomed to the vibrant neon hues of New Mumbai's cityscape, struggled to adjust to the dimly lit landscape
 before her. The air reeked of smoke, ozone, and desperation.

As she stepped out of the makeshift medical tent, the chill of the night air enveloped her, carrying with it the whispers
 of the devastated. Meera's gaze wandered across the sea of makeshift shelters, the rubble-strewn streets, and the twisted
 wreckage of what once were homes. The weight of her actions in the Nexus crushed her, threatening to consume her very being
.

"Meera, we need to talk," a low, urgent voice called out from beside her.

She turned to face Kaito 'Sparrow' Hernandez, leader of the underground rebellion against the corporations exploiting Eon
. His bright hazel eyes, usually ablaze with determination, now seemed dulled by the exhaustion and grief etched 

:white_check_mark: Saved story progress!

##### Generating Scene: Confronting the Architect

The air was alive with the pulse of distorted code, each step echoing through the Architect's Domain like a challenge to the
 very fabric of reality. Meera's heart raced in tandem, her senses on high alert as she led her team deeper into the ever
-shifting labyrinth. Jax flanked her left, his eyes scanning the surroundings with a mixture of fascination and wariness,
 while Dr. Zhang and Maya brought up the rear, their faces set in determined lines.

As they turned a corner, the landscape before them dissolved into a sea of glitch-art constructs, each one a mesmerizing dance
 of light and color. The team's footsteps slowed, awestruck by the sheer scale of the display. Meera, however, pressed on
, her gaze fixed on the figure waiting at the far end of the shimmering expanse.

The Architect, Erebus, stood poised before a backdrop of churning code, its androgynous form shifting subtly with each passing
 moment. The being's presence was both captivating and unnerving, like gazing into the h

:white_check_mark: Saved story progress!

In [ ]:
# from model import StoryDialog, ActDialog, SceneDialog, DialogueLine

# def generate_complete_manuscript(story):
#     # Initialize manuscript content
#     manuscript = StoryDialog(act_dialogues=[])
    
#     # Loop over acts
#     for act_index, act in enumerate(story.acts):
#         act_dialog = ActDialog(act_id=act.act_id, scene_dialogues=[])
        
#         # Loop over scenes in each act
#         for scene_index, scene in enumerate(act.scenes):
#             scene_dialog = SceneDialog(scene_id=scene.scene_id, dialogues=[])
#             last_dialogue_line = ""  # Track the last dialogue line for context
#             scene_summary = f"Scene '{scene.title}': {scene.description}"  # Initial scene context
            
#             while True:
#                 # Generate dialogue for the current scene with updated context
#                 response = query_engine.query(
#                     f"Provide the next dialogue line in JSON format for the scene."
#                     f"\nScene details: {json.dumps(scene.model_dump())}"
#                     f"\nWhat has happened so far: {scene_summary}"
#                     f"\nLast dialogue: '{last_dialogue_line}'"
#                     f"\nEnsure JSON format: {{'character_nickname': '...', 'line': '...'}} or 'Scene Complete'"
#                 )
                
#                 # Check if LLM indicates the end of the scene
#                 if response.response.strip() == "Scene Complete":
#                     print(f"Scene '{scene.title}' generation complete.")
#                     break
                
#                 # Parse JSON response and handle any parsing errors
#                 try:
#                     dialogue_data = json.loads(response.response)
#                     character = dialogue_data["character_nickname"]
#                     text = dialogue_data["line"]
#                 except (json.JSONDecodeError, KeyError, ValueError):
#                     print("Unexpected format in response. Skipping.")
#                     continue
                
#                 # Create and store the new dialogue line
#                 dialogue_line = DialogueLine(character_nickname=character.strip(), line=text.strip())
#                 scene_dialog.dialogues.append(dialogue_line)
                
#                 # Update the context
#                 last_dialogue_line = f"{character}: {text}"
#                 scene_summary += f"\n{character}: {text}"
                
#                 # Update index with the new dialogue line for future context
#                 index.insert_nodes([f"{character}: {text}"])

#             # Add generated scene dialog to act
#             act_dialog.scene_dialogues.append(scene_dialog)
        
#         # Add generated act dialog to manuscript
#         manuscript.act_dialogues.append(act_dialog)
    
#     return manuscript


## Setup llamaindex

In [ ]:
# %pip install llama-index-core

# if settings.TEXT_MODEL_BACKEND == "ollama":
#     %pip install llama-index-embeddings-ollama

In [ ]:
# from IPython.display import Markdown, display
# from model_text import display_messages
# from llama_index.core.llms import ChatMessage
# from utils import blank_story, deindent
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from model_text import llm, llm_json, embedding

# # Create an empty index to store generated content
# index = VectorStoreIndex([], embed_model=embedding)
# query_engine = index.as_query_engine(llm=llm)

In [ ]:
# # Index the story content for context
# from llama_index.core import Document

# def story_to_documents(story):
#     documents = []
    
#     # Add overall story information
#     documents.append(Document(text=f"Title: {story.title}\nGenre: {story.genre}\nMedium: {story.medium}\nVisual Style: {story.visual_style}\nTime Period: {story.time_period}\nLocation: {story.location}\nPlot Overview: {story.plot_overview}\nNarrative Perspective: {story.narrative_perspective}"))
    
#     # Add characters
#     for character in story.characters:
#         char_doc = f"Character: {character.name}\nRole: {character.role}\nDescription: {character.description}\nPersonality: {character.personality}\nPhysical Appearance: {character.physical_appearance}\nCatch Phrase: {character.catch_phrase}\nInternal Conflict: {character.internal_conflict}"
#         documents.append(Document(text=char_doc))
    
#     # Add acts and scenes
#     for act in story.acts:
#         act_doc = f"Act: {act.title}\nDescription: {act.description}"
#         documents.append(Document(text=act_doc))
#         for scene in act.scenes:
#             scene_doc = f"Scene: {scene.title}\nDescription: {scene.description}\nSetting: {scene.setting}\nMood: {scene.mood}"
#             documents.append(Document(text=scene_doc))
    
#     # Add story beats
#     for beat in story.story_beats:
#         beat_doc = f"Story Beat: {beat.name}\nDescription: {beat.description}\nScene: {beat.scene}"
#         documents.append(Document(text=beat_doc))
    
#     # Add subplots
#     for subplot in story.subplots:
#         subplot_doc = f"Subplot: {subplot.title}\nDescription: {subplot.description}"
#         documents.append(Document(text=subplot_doc))
    
#     # Add emotional arc
#     for arc in story.emotional_arc:
#         arc_doc = f"Emotional Arc Stage: {arc.stage}\nDescription: {arc.description}"
#         documents.append(Document(text=arc_doc))
    
#     # Add themes and motifs
#     themes_doc = f"Themes: {', '.join(story.themes)}\nMotifs: {', '.join(story.motifs)}\nConflict Type: {story.conflict_type}"
#     documents.append(Document(text=themes_doc))
    
#     return documents

In [ ]:
# docs = story_to_documents(story)
# for doc in docs:
#     index.insert_nodes([doc])

In [ ]:
# from model import StoryDialog, ActDialog, SceneDialog, DialogueLine

# def generate_dialogue_line(scene_context):
#     response = query_engine.query(f"Generate a dialogue line for the scene: {scene_context}")
#     character, text = response.response.split(": ", 1)
#     return DialogueLine(character=character.strip(), text=text.strip())

# def generate_scene(act_context):
#     response = query_engine.query(f"Generate a scene description for the act: {act_context}")
#     scene = SceneDialog(scene_description=response.response, dialogues=[])
    
#     for _ in range(3):  # Generate 3 dialogue lines per scene
#         scene.dialogues.append(generate_dialogue_line(scene.scene_description))
    
#     return scene

# def generate_act(story_context):
#     response = query_engine.query(f"Generate an act description for the story: {story_context}")
#     act = ActDialog(act_description=response.response, scenes=[])
    
#     for _ in range(3):  # Generate 3 scenes per act
#         act.scenes.append(generate_scene(act.act_description))
    
#     return act

# def generate_story(title):
#     story = StoryDialog(title=title, acts=[])
    
#     for _ in range(3):  # Generate 3 acts
#         act = generate_act(story.title)
#         story.acts.append(act)
        
#         # Add generated content to the index for context
#         index.insert_nodes([act.act_description])
#         for scene in act.scenes:
#             index.insert_nodes([scene.scene_description])
#             for dialogue in scene.dialogues:
#                 index.insert_nodes([f"{dialogue.character}: {dialogue.text}"])
    
#     return story

In [ ]:
# def generate_complete_manuscript(story):
#     manuscript = StoryDialog(act_dialogues=[])
    
#     for act_index, act in enumerate(story.acts):
#         act_dialog = ActDialog(act_id=act.act_id, scene_dialogues=[])
        
#         for scene_index, scene in enumerate(act.scenes):
#             scene_dialog = SceneDialog(scene_id=scene.scene_id, dialogues=[])
#             last_dialogue_line = ""
            
#             while True:
#                 response = query_engine.query(
#                     f"Continue the dialogue for scene '{scene.description}'. "
#                     f"Previous dialogue: '{last_dialogue_line}'. "
#                     f"Generate the next dialogue line using the format 'Character: Dialogue'. "
#                     f"Ensure you use a colon (:) to separate the character name from their dialogue. "
#                     f"If the scene is complete, end your response with 'Scene complete'."
#                 )
                
#                 if "Scene complete" in response.response or len(scene_dialog.dialogues) >= 10:
#                     print(f"Scene '{scene.description}' generation complete.")
#                     break
                
#                 try:
#                     character, text = response.response.split(": ", 1)
#                 except ValueError:
#                     print("Unexpected format in dialogue line. Skipping.")
#                     continue
                
#                 dialogue_line = DialogueLine(character_nickname=character.strip(), line=text.strip())
#                 scene_dialog.dialogues.append(dialogue_line)
                
#                 last_dialogue_line = f"{character}: {text}"
                
#                 index.insert_nodes([f"{character}: {text}"])

#             act_dialog.scene_dialogues.append(scene_dialog)
        
#         manuscript.act_dialogues.append(act_dialog)
    
#     return manuscript

In [ ]:
# manuscript = generate_complete_manuscript(story)

In [ ]:
# dir(index)
# index.summary

# Convert to Markdown and LaTeX

## Load the manuscript

In [3]:
import settings
from model import Story

story = Story.load_from_directory(settings.STORY_DIR + "/step_14")

In [4]:
def story_to_markdown(story):
    """
    Converts a Story and its associated StoryDialog to a Markdown manuscript, formatted like a novel with chapter pages.
    """
    # Start with the title page
    markdown_content = f"# {story.title}\n\n"
    
    # Loop over each act and scene to create the story content
    for act_index, (act, act_dialog) in enumerate(zip(story.acts, story.get_story_dialogue().act_dialogues)):
        # Add a chapter title page for each act
        markdown_content += f"\n\n# Chapter {act_index + 1}\n\n"
        markdown_content += f"_{act.description}_\n\n"  # Italicize the chapter description for a thematic touch
        markdown_content += "<div style='page-break-after: always;'></div>\n\n"  # Page break for chapter separation
        
        # Loop over each scene in the act
        for scene_index, (scene, scene_dialog) in enumerate(zip(act.scenes, act_dialog.scene_dialogues)):
            markdown_content += f"\n\n---\n\n"  # Separator for scenes
            
            # Optional: Add scene description for context
            markdown_content += f"_{scene.description}_\n\n" if scene.description else ""
            
            # Add each dialogue line for the scene
            for dialogue_line in scene_dialog.dialogues:
                # Dialogue formatted as prose
                markdown_content += f"**{dialogue_line.character_nickname}**: {dialogue_line.line}\n\n"
    
    return markdown_content


In [5]:
# Render the markdown
import os

markdown_content = story_to_markdown(story)

In [6]:
# Save it
os.makedirs(f"{settings.STORY_DIR}/step_14", exist_ok=True)
manuscript_md_path = f"{settings.STORY_DIR}/step_14/manuscript.md"

# Save to a Markdown file
with open(manuscript_md_path, "w") as file:
    file.write(markdown_content)

In [7]:
import markdown2
import pypandoc

def markdown_to_latex(markdown_text):
    """
    Convert markdown text to LaTeX using markdown2 and pypandoc.
    """
    # Convert markdown to HTML first
    html_text = markdown2.markdown(markdown_text)
    # Convert HTML to LaTeX
    latex_text = pypandoc.convert_text(html_text, 'latex', format='html')
    return latex_text


In [ ]:
# This one is for plaintext book

# def story_to_latex_novel(story):
#     """
#     Converts a Story and its associated StoryDialog to a LaTeX manuscript formatted as a novel with chapters.
#     """
#     # Start with document preamble
#     latex_content = r"""
#     \documentclass[12pt]{book}
#     \usepackage{setspace}
#     \usepackage{titlesec}
#     \titleformat{\chapter}[display]
#       {\normalfont\huge\bfseries}{\chaptername\ \thechapter}{20pt}{\Huge}
#     \begin{document}
#     \onehalfspacing
#     """

#     # Title page
#     latex_content += f"\\title{{{story.title}}}\n"
#     latex_content += "\\maketitle\n\n"

#     # Loop over each act and scene to create the story content
#     for act_index, (act, act_dialogue) in enumerate(zip(story.acts, story.get_story_dialogue().act_dialogues)):
#         # Add a chapter for each act
#         latex_content += f"\\chapter*{{Chapter {act_index + 1}}}\n"
#         latex_content += f"\\addcontentsline{{toc}}{{chapter}}{{Chapter {act_index + 1}}}\n"
        
#         # Optional: Add a new page for the chapter's start
#         latex_content += "\\newpage\n\n"
        
#         # Loop over each scene in the act
#         for scene_index, (scene, scene_dialogue) in enumerate(zip(act.scenes, act_dialogue.scene_dialogues)):
#             # Add scene title as a section
#             latex_content += f"\\section*{{{scene.title}}}\n"
#             # Optional scene description in italics
#             # if scene.description:
#             #     latex_content += f"\\textit{{{scene.description}}}\n\n"
            
#             # Use the content field for the full scene dialogue and actions, converting from markdown to latex
#             if scene_dialogue.content:
#                 latex_scene_content = markdown_to_latex(scene_dialogue.content)
#                 latex_content += f"{latex_scene_content}\n\n"
            
#             # Add a small space between scenes
#             latex_content += "\\vspace{1em}\n\n"

#         # Insert page break after each chapter
#         latex_content += "\\newpage\n\n"

#     latex_content += "\\newpage\n\n"

#     # Add "Made with Plotomatic" statement at the end
#     latex_content += r"""
#     \vfill
#     \begin{center}
#     \textit{Made with \textbf{Plotomatic}.} \\
#     For more information, visit: \url{https://github.com/mattwilliamson/Plotomatic}
#     \end{center}
#     """

#     # End the document
#     latex_content += "\\end{document}"
    
#     return latex_content


In [69]:
def story_to_latex_graphic_novel(story, scene_image_dir):
    """
    Converts a Story and its associated StoryDialog to a LaTeX manuscript formatted as a graphic novel,
    where each scene is treated as a chapter and may include an image.
    """
    # Start with document preamble
    latex_content = r"""
    \documentclass[12pt]{book}
    \usepackage{setspace}
    \usepackage{titlesec}
    \usepackage{graphicx}
    \usepackage{url}
    \titleformat{\chapter}[display]
      {\normalfont\huge\bfseries}{\chaptername\ \thechapter}{20pt}{\Huge}
    \begin{document}
    \onehalfspacing
    """

    # Title page with title image if available
    title_image_path = os.path.join(scene_image_dir, "title.png")
    if os.path.exists(title_image_path):
        latex_content += f"\\begin{{center}}\n\\includegraphics[width=\\textwidth]{{{title_image_path}}}\n\\end{{center}}\n\\newpage\n\n"
    latex_content += f"\\title{{{story.title}}}\n"
    latex_content += "\\maketitle\n\n"

    # Loop over each scene to create the story content
    chapter_counter = 1
    for act, act_dialogue in zip(story.acts, story.get_story_dialogue().act_dialogues):
        for scene, scene_dialogue in zip(act.scenes, act_dialogue.scene_dialogues):
            # Each scene is a chapter
            latex_content += f"\\chapter*{{Chapter {chapter_counter}: {scene.title}}}\n"
            latex_content += f"\\addcontentsline{{toc}}{{chapter}}{{Chapter {chapter_counter}: {scene.title}}}\n"
            chapter_counter += 1

            # Optional scene description in italics
            if scene.description:
                latex_content += f"\\textit{{{scene.description}}}\n\n"
            
            # Include scene image if it exists
            scene_image_path = os.path.join(scene_image_dir, f"{scene.scene_id}.live.png")
            if os.path.exists(scene_image_path):
                latex_content += f"\\begin{{center}}\n\\includegraphics[width=0.8\\textwidth]{{{scene_image_path}}}\n\\end{{center}}\n\n"

            # Use the content field for the full scene dialogue and actions, converting from markdown to latex
            if scene_dialogue.content:
                latex_scene_content = markdown_to_latex(scene_dialogue.content)
                latex_content += f"{latex_scene_content}\n\n"
            
            # Add a page break after each chapter
            latex_content += "\\newpage\n\n"

    # Add "Made with Plotomatic" statement at the end
    latex_content += r"""
    \vfill
    \begin{center}
    \textit{Made with \textbf{Plotomatic}.} \\
    For more information, visit: \url{https://github.com/mattwilliamson/Plotomatic}
    \end{center}
    """

    # End the document
    latex_content += "\\end{document}"
    
    return latex_content


In [70]:
# latex_content = story_to_latex_novel(story)
latex_content = story_to_latex_graphic_novel(story, settings.STORY_DIR + "/step_6/scenes")
latex_content

'\n    \\documentclass[12pt]{book}\n    \\usepackage{setspace}\n    \\usepackage{titlesec}\n    \\usepackage{graphicx}\n    \\usepackage{url}\n    \\titleformat{\\chapter}[display]\n      {\\normalfont\\huge\\bfseries}{\\chaptername\\ \\thechapter}{20pt}{\\Huge}\n    \\begin{document}\n    \\onehalfspacing\n    \\begin{center}\n\\includegraphics[width=\\textwidth]{stories/my_story/step_6/scenes/title.png}\n\\end{center}\n\\newpage\n\n\\title{Beyond the Pixel Veil}\n\\maketitle\n\n\\chapter*{Chapter 1: The Underdog\'s Rise}\n\\addcontentsline{toc}{chapter}{Chapter 1: The Underdog\'s Rise}\n\\textit{Meera participates in a high-stakes Eon gaming tournament, showcasing her extraordinary skills.}\n\n\\begin{center}\n\\includegraphics[width=0.8\\textwidth]{stories/my_story/step_6/scenes/gaming_tournament.live.png}\n\\end{center}\n\nThe air was electric with anticipation as Meera \'Midnight\' Singh stepped\nonto the vibrant, neon-lit stage of New Mumbai\'s Cyber Arena. The\ncrowd\'s deafenin

In [ ]:
from IPython.display import display, Markdown

step_path = f"{settings.STORY_DIR}/step_14"
manuscript_json_path = f"{step_path}/manuscript.json"
manuscript_latex_path = f"{step_path}/manuscript.tex"
manuscript_pdf_path = f"{step_path}/manuscript.pdf"

# Save to a LaTeX file
with open(manuscript_latex_path, "w") as file:
    file.write(latex_content)

# !pandoc {manuscript_latex_path} -o {manuscript_pdf_path}
!ls -lah {step_path}
!rm manuscript.log
!pdflatex -interaction=nonstopmode -output-directory={step_path} {manuscript_latex_path}

total 4.2M
drwxrwxr-x  2 matt matt 4.0K Nov 12 14:13 .
drwxrwxr-x 16 matt matt 4.0K Nov 12 11:49 ..
-rw-rw-r--  1 matt matt  281 Nov 12 14:21 manuscript.aux
-rw-rw-r--  1 matt matt 9.7K Nov 12 14:21 manuscript.log
-rw-rw-r--  1 matt matt 1.2K Nov 12 14:00 manuscript.md
-rw-rw-r--  1 matt matt 4.1M Nov 12 14:21 manuscript.pdf
-rw-rw-r--  1 matt matt  27K Nov 12 14:24 manuscript.tex
-rw-rw-r--  1 matt matt  26K Nov 12 13:20 story_dialog.json
-rw-rw-r--  1 matt matt  35K Nov 12 13:02 story.json
rm: cannot remove 'manuscript.log': No such file or directory
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./stories/my_story/step_14/manuscript.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/book.cls
Document Class: book 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/bk12

In [72]:
display(Markdown(f"# :open_book: [Download the manuscript PDF]({manuscript_pdf_path})"))

# :open_book: [Download the manuscript PDF](stories/my_story/step_14/manuscript.pdf)

# Generate Dialog (for screenplays)


In [73]:
# # Few Shot Dialog Examples for dialogues
# from model import StoryDialog, ActDialog, SceneDialog, DialogueLine
# from utils import blank_story_dialog

# # Create a StoryDialog object for the story
# few_shot_dialog_1 = StoryDialog(
#     act_dialogues=[
#         ActDialog(
#             act_id="shadows_secrets",
#             scene_dialogues=[
#                 SceneDialog(
#                     scene_id="murder_alley",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="Another rose, just like before... What are you trying to tell me?"),
#                         DialogueLine(character_nickname="jack", line="Nothing about this feels right. Someone's watching. I can feel it.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="jazz_club",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="So, Miss Blackwood, what do you know about the man who left that rose?"),
#                         DialogueLine(character_nickname="evelyn", line="Detective, some secrets are best left in the dark."),
#                         DialogueLine(character_nickname="jack", line="I don't leave anything in the dark, Miss Blackwood.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="mysterious_witness",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="Who is this? How do you know about the murders?"),
#                         DialogueLine(character_nickname="unknown_caller", line="Be careful, detective. High society doesn't like its secrets exposed."),
#                         DialogueLine(character_nickname="jack", line="Wait—hello? Damn it, they hung up.")
#                     ]
#                 )
#             ]
#         ),
#         ActDialog(
#             act_id="dangerous_liaisons",
#             scene_dialogues=[
#                 SceneDialog(
#                     scene_id="the_ball",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="Enjoying the party, Miss Blackwood? Or is this just a cover for something darker?"),
#                         DialogueLine(character_nickname="evelyn", line="Oh, Detective, you know how to spoil a night."),
#                         DialogueLine(character_nickname="jack", line="I'm not here for fun, Evelyn. I'm here for answers.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="alley_confrontation",
#                     dialogues=[
#                         DialogueLine(character_nickname="thug", line="You keep digging, detective, you'll find more than you bargained for."),
#                         DialogueLine(character_nickname="jack", line="I don't scare easy."),
#                         DialogueLine(character_nickname="thug", line="Consider this a warning. Back off the case.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="evelyn_confession",
#                     dialogues=[
#                         DialogueLine(character_nickname="evelyn", line="I didn't want you to know... but I'm part of this mess."),
#                         DialogueLine(character_nickname="jack", line="Then help me clean it up, Evelyn. Together."),
#                         DialogueLine(character_nickname="evelyn", line="You don't understand, Jack. I'm in too deep.")
#                     ]
#                 )
#             ]
#         ),
#         ActDialog(
#             act_id="the_killers_web",
#             scene_dialogues=[
#                 SceneDialog(
#                     scene_id="secret_hideout",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="This isn't just murder. It's a game to them."),
#                         DialogueLine(character_nickname="jack", line="And look here… Evelyn's in this photograph? Damn it, what have you gotten yourself into?")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="ballroom_betrayal",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="Evelyn, tell me the truth. How are you connected to this?"),
#                         DialogueLine(character_nickname="evelyn", line="I never meant for any of this to happen, Jack."),
#                         DialogueLine(character_nickname="killer", line="Always poking where you shouldn't, Detective. Time to end this game."),
#                         DialogueLine(character_nickname="jack", line="I don't back down. Not now, not ever.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="rooftop_showdown",
#                     dialogues=[
#                         DialogueLine(character_nickname="killer", line="You're too late, Holden. This city is mine."),
#                         DialogueLine(character_nickname="jack", line="Not if I have anything to say about it."),
#                         DialogueLine(character_nickname="jack", line="Justice isn't yours to twist. It's time you faced it.")
#                     ]
#                 )
#             ]
#         ),
#         ActDialog(
#             act_id="final_revelation",
#             scene_dialogues=[
#                 SceneDialog(
#                     scene_id="unmasking_truth",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="You don't act alone, do you? The power behind you… they'll pay."),
#                         DialogueLine(character_nickname="killer", line="They're untouchable, Detective. You're nothing against them."),
#                         DialogueLine(character_nickname="jack", line="Maybe, but justice has a way of finding everyone.")
#                     ]
#                 ),
#                 SceneDialog(
#                     scene_id="farewell_with_evelyn",
#                     dialogues=[
#                         DialogueLine(character_nickname="jack", line="So, this is goodbye."),
#                         DialogueLine(character_nickname="evelyn", line="Goodbye, Jack. Maybe, in another life..."),
#                         DialogueLine(character_nickname="jack", line="In another life. Be safe, Evelyn.")
#                     ]
#                 )
#             ]
#         )
#     ]
# )


# few_shot_dialogues = [
#     blank_story_dialog,
#     few_shot_dialog_1,
# ]